<a href="https://colab.research.google.com/github/Murcha1990/ML_AI25/blob/main/Hometasks/Base/HW1_Regression_with_inference_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание №1 (base). Часть 1**

В этом домашнем задании вам будет необходимо:
*  обучить модель регрессии для предсказания стоимости автомобилей;

**Максимальная оценка за дз**
> Оценка за первую часть домашки = $min(\text{ваш балл}, 7)$


**Примечание**

В каждой части оцениваются как код, **так и ответы на вопросы.**

Если нет одного и/или другого, то часть баллов за соответствующее задание снимается.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
from IPython.display import display

random.seed(42)
np.random.seed(42)

Давайте зафиксируем важный момент.

**Задание 0 (0 баллов).**
Изучите и ответье на вопрос: для чего фиксируем сиды в домашках?

**Ответ:** Чтобы результаты были воспроизводимы и не менялись при каждом перезапуске ноутбука. Ассистенту будет проще проверять домашки с одинаковым сидом, на тех же данных.

# **Часть 1 | EDA и визуализация**

Первая часть состоит из классических шагов EDA:

- Базовый EDA и обработка признаков (2.5 балла)
- Визуализации признаков и их анализ (1 балл)

Всего можно набрать 3.5 основных балла и 0.65 бонусных. Бонусные задания выделены как **Дополнительное задание/Бонус**. Вы можете выполнять их, чтобы в случае ошибок в основных задачах всё равно набрать за работу максимум. Кроме того, дополнительные задания позволяют вам углубить знания.

Призываем активно использовать их!

## **Простейший EDA и обработка признаков (2.5 балла)**

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

### **Задание 1 (0.5 балла)**

Вы уже встречались с pandas в первой половине семестра. Теперь будем постоянно (кроме некоторых случаев) использовать его для анализа данных и наслаивать навыки. Выполните операции, направленные на практику основных действий с `pandas`:

**0.1 балла**
- [ ] Отобразите 30 случайных строк тренировочного датасета.
- [ ] Отобразите первые 5 и последние 5 объектов тестового датасета
- [ ] Посмотрите, есть ли в датасете пропуски. Запишите/выведите названия колонок, для которых есть пропущенные значения
- [ ] Посмотрите, есть ли в данных явные дубликаты


**0.2 балла**
Ответьте на вопросы:
- [ ] Выводы о чем можно сделать, используя случайные/верхние/нижние строки? А о чем нельзя?
- [ ] Достаточно ли метода duplicated для анализа дубликатов? Почему?

**0.15 балла**
- [ ] Постройте дашборд, используя [ydata-profilling](https://github.com/ydataai/ydata-profiling)

**Ответ:** Случайные строки показывают разнообразие данных. Верхние/нижние - порядок сортировки в источнике. Нельзя судить о статистиках распределений. duplicated() ищет только полные дубликаты строк, не учитывает частичные совпадения.


In [ ]:
display(df_train.sample(30, random_state=42))

display(pd.concat([df_test.head(5), df_test.tail(5)]))

display((
    df_train.columns[df_train.isna().any()].tolist(),
    df_test.columns[df_test.isna().any()].tolist()
))

display((
    df_train.duplicated().sum(),
    df_test.duplicated().sum()
))

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df_train, title="Train Data Profile", explorative=True)
profile.to_notebook_iframe()

Мы обнаружили пропуски. Давайте избавимся от них.

**(0.05 балла)**
- [ ] Заполните пропуски в столбцах медианами. Убедитесь, что после заполнения пропусков не осталось. Заполнение пропусков проводите для обоих наборов данных, если необходимо

**Важно!**

При заполнении пропусков и в тестовом, и тренировочном наборах данных вы определяетесь со стратегией предобработки пропущенных значений при потенциальной работе модели.

Так как в теоретическом случае вы не имеете доступа к тестовой выборке, то заполняемой значение (у нас — медиана) вы считаете про *тренировочному* набору данных и им же заполняете *тестовый*.

In [ ]:
numeric_cols = df_train.select_dtypes(include=[np.number]).columns

medians = df_train[numeric_cols].median()

df_train[numeric_cols] = df_train[numeric_cols].fillna(medians)
df_test[numeric_cols] = df_test[numeric_cols].fillna(medians)

df_train.isna().sum().sum(), df_test.isna().sum().sum()

### **Задание 2 (0.5 балла)**

На прошлом шаге вы рассмотрели дубликаты. Однако дубликат может быть связан с не только полным повторением информации в нескольких строках, но и частиным. Например, объект мог был внесен в базу данных с разным значением целевой переменной. В этом шаге займемся такими дублями!

**Ваши действия:**

- [ ] Посмотрите, есть ли в трейне объекты с одинаковым признаковым описанием (целевую переменную следует исключить). Если есть, то сколько? (0.1 балла)
- [ ] Отобразите такие объекты (0.15 балла)
- [ ] Удалите повторяющиеся строки. Если при одинаковом признаковом описании цены на автомобили отличаются, то оставьте первую строку по этому автомобилю (0.15 балла)
- [ ]  Обновите индексы строк таким образом, чтобы они шли от 0 без пропусков (0.1 балла)


In [ ]:
target_col = "selling_price"
feature_cols = df_train.columns.drop(target_col)

dup_mask = df_train.duplicated(subset=feature_cols, keep=False)
n_dup_rows = int(dup_mask.sum())
print("Дубликаты по признакам (строки):", n_dup_rows)

display(df_train.loc[dup_mask].sort_values(list(feature_cols)))

df_train = (
    df_train
    .drop_duplicates(subset=feature_cols, keep="first")
    .reset_index(drop=True)
)

print("Итоговая форма:", df_train.shape)

In [ ]:
assert df_train.shape == (5840, 13)

Отлично! Мы избавились от маленьких и явных проблем. Теперь перейдем к более сложным недостаткам полученной таблицы.

### **Задание 3 (0.25 балла)**

Вы могли заметить, что с признаками ``mileage, engine, max_power и torque`` всё не очень хорошо. Они распознаются как строки (можно убедиться в этом, вызвав `data.dtypes`). Однако эти переменные не являются категориальными — они — числа. Соответственно, нужно привести их к числовому виду.

**Задача :**
* [ ] Уберите единицы измерения для признаков ``mileage, engine, max_power``.
* [ ] Приведите тип данных к ``float``.
* [ ] Удалите столбец ``torque``


**Важно**
- Все действия нужно производить над обоими датасетами — `train` и `test`.
- Стобец ``torque`` мы удаляем для простоты. В идеальном случае, его также стоило бы предобработать.

In [ ]:
cols_to_float = ["mileage", "engine", "max_power"]

for df in (df_train, df_test):
    for col in cols_to_float:
        df[col] = pd.to_numeric(
            df[col].astype(str).str.replace(r"[^0-9.]+", "", regex=True),
            errors="coerce"
        ).astype("float")
    df.drop(columns=["torque"], inplace=True, errors="ignore")

df_train[cols_to_float].dtypes, df_test[cols_to_float].dtypes, df_train.shape, df_test.shape

### **Задание 4 (0.1 балла)**

Теперь, когда не осталось пропусков, давайте преобразуем столбцы к более подходящим типам. А именно столбцы ``engnine`` и ``seats`` к приведем к `int`.

- [ ] Осуществите приведение столбцов к необходимому типу.

In [ ]:
cols_to_int = ["engine", "seats"]

fill_vals = df_train[cols_to_int].median()

for df in (df_train, df_test):
    df[cols_to_int] = df[cols_to_int].fillna(fill_vals)
    df[cols_to_int] = df[cols_to_int].round().astype("int64")

df_train[cols_to_int].dtypes, df_test[cols_to_int].dtypes

### **Задание 5 (0.15 балла)**

Отлично! Мы провели "косметическую" предобработку и теперь готовы сделать важный шаг в контексте анализа данных. А именно — посмотреть на статистики!

**Ваша задача:**
- [ ] Посчитайте основные статистики по числовым столбцам для трейна и теста
- [ ] Посчитайте основные статистики по категориальным столбцам для трейна и теста
- [ ] Сделайте вывод.

**Подсказка:**

Используте ``.describe()`` с нужным(и) аргументом(-ами).

**Примечание:**

Более корректно рассматривать статистики до заполнения пропусков и после, чтобы убедиться, что мы не внесли каких-либо серьезных сдвигов в изначальные рапсределения.

In [ ]:
print("TRAIN — числовые")
display(df_train.describe(include=[np.number]))

print("TRAIN — категориальные")
display(df_train.describe(include=["object"]))

print("TEST — числовые")
display(df_test.describe(include=[np.number]))

print("TEST — категориальные")
display(df_test.describe(include=["object"]))

**Вывод:** Распределения train и test похожи. Медианы и среднее близки. Категориальные признаки имеют доминирующие категории


In [ ]:
assert df_train.shape[0] == 5840

## **Визуализации (1 балл + 0.5 бонус)**



Визуализация данных — важный шаг в работе. Визуализировать данные необходимо, например, чтобы:

- Оценить распределения признаков самих по себе (это может натоклнуть вас на мысли о модели, которую можно использовать)
- Сравнить распределения на `train` и `test` — чтобы проверить, насколько информация, на которой вы будете обучаться согласуется с той, на которой модель должна работать
- Оценить есть ли явная связь признаков с целевой переменной

**Важно:**

Если распределения на `train` и `test` не совпадают, это не значит, что нужно перемешивать данные! Более корректно актуализировать задачу и уточнить, а не устарели ли данные `train`. Также полезным может быть собрать новую тестовую выборку, смешав те, что имеются сейчас.

**Если вы будете подгонять распределения, то можете встретиться с переобучением!**

### **Задание 6 (0.5 балла)**

Шаг 1.
- [ ] Воспользуйтесь `pairplot` из библиотеки `seabron`, чтобы визуализировать попарные распределения числовых признаков для `train`
- [ ] По полученному графику ответьте на вопросы:
 - Можно ли предположить на основе распределений связь признаков с целевой переменной?
 - Можно ли предположить на основе распределений выдвинуть гипотезу о корреляциях признаков?

Шаг 2.

- [ ] Постройте pairplot по тестовым данным
- [ ] Ответьте на вопрос "Похожими ли оказались совокупности при разделении на трейн и тест?"

In [ ]:
%matplotlib inline

for name, df in [("TRAIN", df_train), ("TEST", df_test)]:
    num_cols = df.select_dtypes(include=[np.number]).columns
    g = sns.pairplot(df[num_cols], diag_kind="kde", plot_kws={"alpha": 0.6, "s": 10})
    g.fig.suptitle(name, y=1.02)

**Ответ:** Да. По pairplot видно, что selling_price заметно связан с year, engine и max_power (чем новее авто и выше характеристики, тем выше цена), а с km_driven связь слабая отрицательная. Также можно выдвинуть гипотезы о корреляциях между признаками: engine и max_power сильно положительно коррелируют между собой, year отрицательно связан с km_driven (старые машины обычно имеют больший пробег)

### **Задание 7 (0.5 балла)**

И так, вы выдвинули гипотезы о наличии связи. Теперь давайте оценим эту связь в числах.

**Задание:**
- [ ] Получите значения коэффициента корреляции Пирсона для тренировочного набора данных при помощи `pd.corr()`
- [ ] По полученным корреляциям постройте тепловую карту (`heatmap` из бибилотеки seaborn)


In [ ]:
num = df_train.select_dtypes(include=[np.number])
corr = num.corr(method="pearson")

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", square=True)
plt.show()

- [ ] Ответьте на вопросы:
 - Какие 2 признака наименее скоррелированы между собой?
 - Между какими наблюдается довольно сильная положительная линейная зависимость?
 - Правильно ли, опираясь на данные, утверждать, что чем меньше год, тем, скорее всего, больше километров проехала машина к дате продажи?

**Ответ:** Наименее связаны между собой year и engine, корреляция почти нулевая. Сильная положительная связь есть между selling_price и max_power, также заметна между engine и max_power. Утверждать, что чем меньше год выпуска, тем больше пробег, в целом можно: между year и km_driven есть заметная отрицательная связь

### **Бонус (0.5 балла)**

Если вам кажется, что мы не попросили вас нарисовать какие-то очень важные зависимости, нарисуйте их **и поясните.**

- Каждая дополнительная визуализация может принести до 0.25 баллов при условии, что она обоснована.

In [ ]:
c = corr.where(~np.eye(corr.shape[0], dtype=bool)).stack()

least_pair = c.abs().idxmin()
least_val = float(c.loc[least_pair])

strong_pair = c.idxmax()
strong_val = float(c.max())

year_km = float(corr.loc["year", "km_driven"]) if {"year", "km_driven"} <= set(corr.columns) else None

print("Наименее скоррелированы:", least_pair, "corr=", least_val)
print("Сильная положительная связь:", strong_pair, "corr=", strong_val)
print("corr(year, km_driven) =", year_km)


# **Часть 2 | Модель только на вещественных признаках**

В этой части вам предстоит обучить модель только на вещественных признаках. Почему только на них?

Чем больше признаковое пространство — чем сложнее модель. А чем модель проще — тем лучше для скорости работы и интерпретации признаков.

За задания этой части вы можете набрать 1.25 балла;

### **Задание 8 (0.05 балла)**

Сделайте тренировочный и тестовый наборы. Сделайте на тренировочный и тестовый наборы. Она уже даны, достаточно просто отделить целевой признак Перед разбиением создайте копию датафрейма, который будет хранить только вещественные признаки и используйте его (то есть категориальные столбцы (все, кроме seats) необходимо удалить).

В переменные y_train и y_test запишите значения целевых переменных. Используйте уже имеющиеся данные train и test при разбиении. Если считаете уместным применить какую-то более хитрую технику разбиения — примените, но опишите, зачем.

In [ ]:
target = "selling_price"

y_train = df_train[target].copy()
y_test = df_test[target].copy()

X_train = df_train.drop(columns=[target]).select_dtypes(include=[np.number]).copy()
X_test = df_test.drop(columns=[target]).select_dtypes(include=[np.number]).copy()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.columns.tolist()

Использованы вещественные признаки:

year - год выпуска автомобиля, km_driven - пробег, mileage - расход топлива, engine - объем двигателя, max_power - мощность двигателя, seats - количество мест

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

medians = X_train.median()
X_train = X_train.fillna(medians)
X_test = X_test.fillna(medians)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

r2_train, r2_test

### **Задание 9 (0.2 балла)**

Построим нашу первую модель!
- [ ] Обучите классическую линейную регрессию с дефолтными параметрами. Посчтитайте $R^2$ и $MSE$ для трейна и для теста.
- [ ] Сделайте выводы по значениям метрик качества.

**Примечание:**

Здесь и далее $R^2$ и $MSE$ для трейна и для теста выводите везде, где требуется обучать модели, даже если в явном виде этого не просят. Иначе непонятно, как понять, насколько успешны наши эксперименты.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error as MSE

model = LinearRegression()
model.fit(X_train, y_train)

pred_tr = model.predict(X_train)
pred_te = model.predict(X_test)

r2_tr = r2_score(y_train, pred_tr)
r2_te = r2_score(y_test, pred_te)
mse_tr = MSE(y_train, pred_tr)
mse_te = MSE(y_test, pred_te)

r2_tr, r2_te, mse_tr, mse_te

**Вывод:** Значения R² на train и test почти одинаковые (~0.59), явного переобучения нет. Модель объясняет около 59% вариации цены. Качество среднее, но стабильное. MSE на тесте выше, чем на трейне, что ожидаемо, разрыв умеренный, модель обобщается нормально

### **Задание 10 (0.15 балла)**

Всегда есть место совершенству. Поэтому давайте попробуем улучшить модель. При помощи стандартизации признаков.

- [ ] Стандартизируйте значения в тренировочных и тестовых данных. Стандартизатор **обучайте только на `train`**.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error as MSE

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

model_sc = LinearRegression()
model_sc.fit(X_train_sc, y_train)

pred_tr = model_sc.predict(X_train_sc)
pred_te = model_sc.predict(X_test_sc)

r2_tr = r2_score(y_train, pred_tr)
r2_te = r2_score(y_test, pred_te)
mse_tr = MSE(y_train, pred_tr)
mse_te = MSE(y_test, pred_te)

r2_tr, r2_te, mse_tr, mse_te

### **Задание 11 (0.1 балла)**

Хотя стандартизация не помогла сильно прибавить в качестве она открыла возможность интерпретировать важность признаков в модели. Правило интерпретации такое:

Чем больше коэффициент $\beta_i$ по модулю, тем важнее признак.

**Ответьте на вопрос:**

- [ ] Какой признак оказался наиболее информативным в предсказании цены?

In [ ]:
import numpy as np
import pandas as pd

coef = pd.Series(model_sc.coef_, index=X_train.columns).sort_values(key=np.abs, ascending=False)
coef.head(1)

**Ответ:** Наиболее информативным признаком оказался max_power, так как у него наибольший по модулю коэффициент в стандартизированной линейной модели

### **Задание 12 (0.25 балла)**

Попробуем улучшить нашу модель с помощью применения регуляризации. Для этого воспльзуемся `Lasso` регрессией.  Кроме того, попробуйте использовать её теоретическое свойство отбора признаков, за счет зануления незначимых коэффициентов.

**Задание:**

- [ ] Обучите Lasso регрессию на тренировочном наборе данных с нормализованными признаками. Оцените её качество
- [ ] Проверьте, занулила ли L1-регуляризация с параметрами по умолчанию какие-нибудь веса? Предположите почему.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_squared_error as MSE

lasso = Lasso(random_state=42)
lasso.fit(X_train_sc, y_train)

pred_tr = lasso.predict(X_train_sc)
pred_te = lasso.predict(X_test_sc)

r2_tr, r2_te, mse_tr, mse_te = (
    r2_score(y_train, pred_tr),
    r2_score(y_test, pred_te),
    MSE(y_train, pred_tr),
    MSE(y_test, pred_te),
)

zeros = (lasso.coef_ == 0).sum()
r2_tr, r2_te, mse_tr, mse_te, zeros

In [ ]:
zeros, lasso.alpha

### **Задание 13 Финальный рывок (0.5 балла)**

До этого мы с вами использовали `train` для обучения и `test` для прогнозирования. Но у нас есть ещё одна задача — подобрать оптимальные параметры модели. Для этого используем кросс-валидацию, описанную на семинарах.

Кроме того, выжмем максимум из модификаций регрессии. Построим `ElasticNet`. И сделаем всё по порядку.

**Ваша задача 1:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для Lasso-регрессии. Вам пригодится класс [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
- [ ] Ответьте на вопросы:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Какой коэффициент регуляризации у лучшей из перебранных моделей? Занулились ли какие-нибудь из весов при такой регуляризации?

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

lasso = Lasso(max_iter=20000, random_state=42)

param_grid = {"alpha": np.logspace(-4, 2, 25)}
gs_lasso = GridSearchCV(lasso, param_grid=param_grid, cv=10, scoring="r2", n_jobs=-1)
gs_lasso.fit(X_train_sc, y_train)

best_lasso = gs_lasso.best_estimator_
n_models_lasso = len(gs_lasso.cv_results_["params"])
best_alpha = gs_lasso.best_params_["alpha"]
zeros_lasso = int((best_lasso.coef_ == 0).sum())

n_models_lasso, best_alpha, zeros_lasso

Для Lasso было обучено 25 моделей (по числу значений alpha), лучшая модель имеет alpha ≈ 100, при этом ни один коэффициент не занулился, так как оптимальная регуляризация оказалась слишком слабой для отбора признаков

**Ваша задача 2:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) регрессии.
- [ ] Ответьте на вопрос:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Какие гиперпараметры соответствуют лучшей (по выбранной метрике качества) из перебранных моделей?

In [ ]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet(max_iter=20000, random_state=42)

param_grid = {
    "alpha": np.logspace(-4, 2, 25),
    "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9, 1.0],
}
gs_enet = GridSearchCV(enet, param_grid=param_grid, cv=10, scoring="r2", n_jobs=-1)
gs_enet.fit(X_train_sc, y_train)

n_models_enet = len(gs_enet.cv_results_["params"])
best_params_enet = gs_enet.best_params_

n_models_enet, best_params_enet

Для ElasticNet было обучено 150 моделей (25 значений alpha × 6 значений l1_ratio), лучшая модель соответствует alpha ≈ 0.316 и l1_ratio = 0.5

# **Часть 3| Добавляем категориальные фичи**

Попробуем для улучшения модели дать ей больше признаков. Добавим категориальные фичи.

За эту часть можно набрать 0.75 основных балла и 0.25 бонусных.


**Задание 14 (0.1 балла)** Проанализируйте столбец `name`. Очевидно, что эта переменная является категориальной, однако категорий в ней много.

В этом домашнем задании мы предлагаем удалить его.

**Ваша задача:**
- [ ] Удалить столбец`name`

In [ ]:
target = "selling_price"

X_train_cat = df_train.drop(columns=[target, "name"]).copy()
X_test_cat  = df_test.drop(columns=[target, "name"]).copy()
y_train_cat = df_train[target].copy()
y_test_cat  = df_test[target].copy()

В другом случае, конечно, мы могли бы предобработать данный столбец. В качестве бонуса предлагаем вам придумать и реализовать алгоритм предобработки.

**Бонус 0.25 балла**
- [ ] Предобработайте столбец `name`, чтобы избежать его удаления. Если предобработали столбец — можно закомментировать assert

In [ ]:
assert X_train_cat.shape == (5840, 10)

In [ ]:
X_train_cat.describe(include='object')

### **Задание 15 (0.4 балла)**

- [ ] Закодируйте категориалльные фичи и ``seats`` методом OneHot-кодирования. Обратите внимание, что во избежание мультиколлинеарности следует избавиться от одного из полученных столбцов при кодировании каждого признака методом OneHot.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

num_cols = X_train_cat.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_train_cat.columns.difference(num_cols).tolist() + ["seats"]
num_cols = [c for c in num_cols if c != "seats"]

med = X_train_cat[num_cols].median()
Xtr = X_train_cat.copy()
Xte = X_test_cat.copy()
Xtr[num_cols] = Xtr[num_cols].fillna(med)
Xte[num_cols] = Xte[num_cols].fillna(med)
Xtr[cat_cols] = Xtr[cat_cols].astype("string").fillna("missing")
Xte[cat_cols] = Xte[cat_cols].astype("string").fillna("missing")

ohe = OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False)
Otr = ohe.fit_transform(Xtr[cat_cols])
Ote = ohe.transform(Xte[cat_cols])

X_train_full = pd.concat([Xtr[num_cols].reset_index(drop=True),
                          pd.DataFrame(Otr, columns=ohe.get_feature_names_out(cat_cols))],
                         axis=1)
X_test_full  = pd.concat([Xte[num_cols].reset_index(drop=True),
                          pd.DataFrame(Ote, columns=ohe.get_feature_names_out(cat_cols))],
                         axis=1)

X_train_full.shape, X_test_full.shape

### **Задание 16 (0.25 балла)**
Повторим то, что делали на прошлом шаге для моделей на вещественных признаках, однако теперь с моделью `Ridge`.


**Ваша задача:**
- [ ] Переберите параметр регуляризации `alpha` для гребневой (ridge) регрессии с помощью класса `GridSearchCV` В качестве параметров при объявлении GridSearchCV кроме модели укажите метрику качества $R^2$. Кроссвалидируйтесь по 10-ти фолдам.
- [ ] Ответье на вопрос: Удалось ли улучшить качество прогнозов?

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

ridge = Ridge(random_state=42)
grid = GridSearchCV(ridge, {"alpha": np.logspace(-4, 4, 25)}, cv=10, scoring="r2", n_jobs=-1)
grid.fit(X_train_full, y_train_cat)

best_ridge = grid.best_estimator_
r2_tr = r2_score(y_train_cat, best_ridge.predict(X_train_full))
r2_te = r2_score(y_test_cat, best_ridge.predict(X_test_full))

grid.best_params_["alpha"], r2_tr, r2_te

**Ответ:** Да, качество улучшилось: R² на test стало выше, переобучения нет

# **Часть 4. | Бизнесовая (0.5 балла)**

### **Задание 17 (0.5 балла)**

В мире бизнеса очень важно давать оценку качества модели понятную бизнесу, поэтому иногда заказчики приходят с кастомными метриками. Попробуем сделать такую для нашей задачи.

**Описание метрики:**

Среди всех предсказанных цен на авто нужно посчитать долю прогнозов, отличающихся от реальных цен на эти авто не более чем на 10% (в одну или другую сторону)

**Ваша задача:**

- [ ] Реализуйте метрику `business_metric`
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какая лучше всего решает задачу бизнеса

In [ ]:
def business_metric(y_true, y_pred):
    return (abs(y_pred - y_true) / y_true <= 0.1).mean()

In [ ]:
bm_linear = business_metric(y_test, model.predict(X_test))
bm_lasso  = business_metric(y_test, best_lasso.predict(X_test_sc))
bm_ridge  = business_metric(y_test_cat, best_ridge.predict(X_test_full))

bm_linear, bm_lasso, bm_ridge

**Ответ:** Лучший результат по бизнес-метрике показывает Ridge (≈0.245), значит именно эта модель чаще всего предсказывает цену с ошибкой не более 10% и лучше всего решает задачу бизнеса

## Сохранение модели и артефактов


In [ ]:
import pickle

model_artifacts = {
    'model': best_ridge,
    'ohe': ohe,
    'num_cols': num_cols,
    'cat_cols': cat_cols,
    'medians': med,
    'feature_names': X_train_full.columns.tolist(),
    'y_train': y_train_cat,
    'X_train_full': X_train_full
}

with open('model_pipeline.pkl', 'wb') as f:
    pickle.dump(model_artifacts, f)

# **<font color="green">Часть 5 | Создание интерактивного приложения на Streamlit (3 балла)</font>**

Вам необходимо создать интерактивное приложение на Streamlit, которое будет:

- Показывать основные информативные графики/гистограммы в рамках EDA (1 балл)
- На вход запрашивать csv-файл с признаками объектов или запрашивать признаки объекта в окошках для ввода, и применять на поступивших объектах модель (1 балл)
- Визуализировать веса обученной модели (1 балл)

# **Часть 6 | Оформление репозитория и оценка сервиса (2 балла)**

**Результаты вашей работы** необходимо разместить в своём GitHub-репозитории. В этот же репозиторий позже будет добавлена вторая часть.

Под результатами первой части понимаем следующее:

---

### Обязательные файлы:

1. **`.ipynb`-ноутбук** со всеми экспериментами:

   * полный EDA,
   * все шаги препроцессинга,
   * обучение и сравнение моделей,
   * сохранённые output’ы.

2. **`.pickle`-файл**, содержащий:

   * обученную модель (или пайплайн `scaler + model`);
   * параметры скейлинга;
   * любые числовые объекты, необходимые для инференса внутри Streamlit-приложения.

3. **`.md`-файл с выводами** о проделанной работе:

   * что было сделано (краткое описание каждого этапа);
   * какие результаты были получены (метрики + интерпретация);
   * что дало наибольший прирост качества;
   * что сделать не удалось и почему (это нормально и даже полезно);
   * **оценка разработанного сервиса**:

     * насколько приложение удобно в использовании;
     * что получилось визуализировать хорошо, а что — менее удачно;
     * какие ограничения или проблемы вы заметили;
     * какие улучшения планируете в следующей итерации.

По результатам второй части, ждем:

4. **Streamlit-приложение** (например, `app.py`), которое можно запустить командой:

   ```bash
   streamlit run app.py
   ```

**И ссылку на приложение в StreamlitHub.**

В приложении должно быть:

* отображение ключевых графиков EDA;
* ввод данных (CSV или ручной ввод);
* применение модели;
* визуализация весов/коэффициентов.



### **За что могут быть сняты баллы:**

* отсутствие `.pickle` с моделью / пайплайном;
* слабая или неполная аналитика в `.md`-файле;
* беспорядок в ноутбуке или неясная логика вычислений;
* отсутствие возможности запустить Streamlit-приложение.